# Concatenating and merging data

Functions to concatenate or merge DataFrames in pandas:

1. [`pd.concat()`](https://pandas.pydata.org/docs/reference/api/pandas.concat.html): combine multiple DataFrames by appending observations (rows) or columns
2. [`pd.merge()`](https://pandas.pydata.org/docs/reference/api/pandas.merge.html): match observations from one DataFrame with observations from another DataFrame

You can also consult the official 
[user guide](https://pandas.pydata.org/docs/user_guide/merging.html) 
and the pandas 
[cheat sheet](https://pandas.pydata.org/Pandas_Cheat_Sheet.pdf) 
for more information.

***
## Concatenation

### Concatenating Series

*Example: Concatenating two Series along the row axis*

- Specify `pd.concat(..., axis=0)` (or use default value)

In [1]:
import pandas as pd

a = pd.Series(["A1","A2","A3"])

data = [f"B{i}" for i in range(5)]

b = pd.Series(data)

b

0    B0
1    B1
2    B2
3    B3
4    B4
dtype: object

*Example: Concatenating along the column axis*

- Specify `pd.concat(..., axis=1)`
- Usually only makes sense for equal-length `Series`
- Explicitly set columns names using `pd.concat(..., keys=...)`

In [2]:
s = pd.concat((a, b), axis = 0).reset_index(drop=True)
s

0    A1
1    A2
2    A3
3    B0
4    B1
5    B2
6    B3
7    B4
dtype: object

<div class="alert alert-info">
<h3> Your turn</h3>
<ol>
    <li>Create a new <TT>Series</TT> with observations <TT>['C1', 'C2']</TT>.</li>
    <li>Using the previously created <TT>Series</TT> <TT>a</TT> and <TT>b</TT>, concatenate all three objects along the row axis and create a new (unique) index.</li>
    <li>Repeat the previous step, but now concatenate along the column axis. Assign the column names <TT>'Column1'</TT>, <TT>'Column2'</TT>, and <TT>'Column3'</TT>.</li>
</ol>
</div>

In [3]:
c = pd.Series(['C1', 'C2'])
c

0    C1
1    C2
dtype: object

In [4]:
s = pd.concat((a, b, c))

***
### Concatenating DataFrames

In [5]:
# 2 x 2 DataFrame
import numpy as np

data_a = np.array([f'A{i+1}' for i in range(4)]).reshape((2, 2))
data_a
df_a = pd.DataFrame(data_a)

In [6]:
data_b = np.array([f'B{i+1}' for i in range(6)]).reshape((2, 3))

df_b = pd.DataFrame(data_b)

#### Concatenating along the column axis

*Example: Concatenating two DataFrames along the column axis*

- Specify `pd.concat(..., axis=1)`

In [7]:
df = pd.concat((df_a, df_b), axis=1)
df

,0,1,0,1,2
0,A1,A2,B1,B2,B3
1,A3,A4,B4,B5,B6


In [8]:
df.columns = [f'Column{i}' for i in range(len(df.columns))]

*Example: Concatenating a DataFrame and a Series*

In [9]:
df = pd.concat((df_a, c), axis=1)
df

,0,1,0
0,A1,A2,C1
1,A3,A4,C2


#### Concatenating along the row axis

*Example: Concatenating rows with identical columns*

In [10]:
df_a

,0,1
0,A1,A2
1,A3,A4


In [11]:
df_b.T

,0,1
0,B1,B4
1,B2,B5
2,B3,B6


In [12]:
df = pd.concat((df_a, df_b.T), axis=0).reset_index(drop=True)
df

,0,1
0,A1,A2
1,A3,A4
2,B1,B4
3,B2,B5
4,B3,B6


*Example: Concatenating rows with different columns*

- Creates `NaN`s for non-overlapping columns

In [13]:
df = pd.concat((df_a, df_b), axis=0)
df

,0,1,2
0,A1,A2,NaN
1,A3,A4,NaN
0,B1,B2,B3
1,B4,B5,B6


<div class="alert alert-info">
<h3> Your turn</h3>
Use the data files located in the folder <TT>../data/FRED</TT> to perform the following tasks:
<ol>
    <li>Load the data in <TT>FRED_monthly_1950.csv</TT> and <TT>FRED_monthly_1960.csv</TT> into two different DataFrames.
        The files contain monthly macroeconomic time series for the 1950s and 1960s, respectively.
        <br/>
        <i>Hint:</i> Use <TT>pd.read_csv(..., parse_dates=['DATE'])</TT> to automatically parse strings stored in the <TT>DATE</TT> column as dates.
        </li>
    <li>Concatenate these DataFrames along the row dimension to get a total of 240 observations.</li>
    <li>Set the column <TT>Date</TT> as index for the newly created DataFrame.</li>
</ol>
</div>

In [14]:
# Path to local data folder
DATA_PATH = '/Users/olavgramstadberstad/Desktop/repos/TECH2-H24/data/FRED'

df1 = pd.read_csv(f'{DATA_PATH}/FRED_monthly_1950.csv', parse_dates=['DATE'])
df1['DATE']

df2 = pd.read_csv(f'{DATA_PATH}/FRED_monthly_1960.csv', parse_dates=['DATE'])
df2['DATE']

df = pd.concat((df1, df2), axis=0)
df['DATE']

df.set_index('DATE')


,CPI,UNRATE,FEDFUNDS,REALRATE,LFPART
DATE,,,,,
1950-01-01,23.5,6.5,NaN,NaN,58.9
1950-02-01,23.6,6.4,NaN,NaN,58.9
1950-03-01,23.6,6.3,NaN,NaN,58.8
1950-04-01,23.6,5.8,NaN,NaN,59.2
1950-05-01,23.8,5.5,NaN,NaN,59.1
...,...,...,...,...,...
1969-08-01,36.9,3.5,9.2,NaN,60.3
1969-09-01,37.1,3.7,9.2,NaN,60.3
1969-10-01,37.3,3.7,9.0,NaN,60.4


***
## Merging and joining data sets

### Types of merges

We can merge DataFrames `A` and `B` in various ways:

1.  *one-to-one*: Unique keys in `A` and `B`
2.  *many-to-one*: Unique keys in `A`, non-unique keys in `B`
3.  *many-to-many*: Non-unique keys in both `A` and `B` (avoid this)

### Implementation in pandas

1.  [`pd.merge()`](https://pandas.pydata.org/docs/reference/api/pandas.merge.html): function that takes as argument the *two* DataFrames to be merged,
    e.g.,
    
    ```python
    result = pd.merge(df_A, df_B)
    ```
2.  [`df.merge()`](https://pandas.pydata.org/docs/reference/api/pandas.DataFrame.merge.html): method of a specific `DataFrame` object, takes the other `DataFrame` as argument, e.g.,
    
    ```python
    result = df_A.merge(df_B)
    ```


### Controlling the resulting data set

Merge methods:

1.  `how='inner'` (_inner join_): _intersection_ of keys that are present in _both_ data sets
2.  `how='outer'` (_outer join_): _union_ of keys present in either of the data sets
3.  `how='left'` (_left join_): all identifiers from the _left_ data set are in the result
4.  `how='right'` (_right join_): all identifiers from the _right_ data set are in the result

Illustration: Each circle presents the keys present in the left (`df1`) or right (`df2`) DataFrames. The merge method controls which subset of keys is retained in the merge result.

![Join types](../lecture4/join-methods.png)

***
### Merging with `merge()`

- Create two DataFrames with partially overlapping keys

In [15]:
df_a = pd.DataFrame({'key': [0, 1], 'value_a' : ['A0', 'A1']})
df_a

,key,value_a
0,0,A0
1,1,A1


In [16]:
df_b = pd.DataFrame({'key': [1, 2], 'value_b' : ['B1', 'B2']})
df_b

,key,value_b
0,1,B1
1,2,B2


#### Using `pd.merge()`

*Example: one-to-one merges*

Merge using the following methods:

1.  `inner` (default for `pd.merge()`)
2.  `outer`
3.  `left`
4.  `right`

In [44]:
df = pd.merge(df_a, df_b, on='key', how='left')
df

,key,value_a,value_b
0,0,A0,NaN
1,1,A1,B1


#### Using `DataFrame.merge()`

In [18]:
df_a.merge(df_b, on='key', how='right')

,key,value_a,value_b
0,1,A1,B1
1,2,NaN,B2


*Example: Merging with overlapping column names*

- pandas automatically renames overlapping column names not used as keys
- Can use `merge(..., suffixes=...)` to override default suffixes for _left_ and _right_

In [19]:
df_a

,key,value_a
0,0,A0
1,1,A1


In [20]:
df_b

,key,value_b
0,1,B1
1,2,B2


In [21]:
df_a = df_a.rename(columns={'value_a': 'value'})
df_b = df_b.rename(columns={'value_b': 'value'})

In [22]:
pd.merge(df_a, df_b, on='key', suffixes=('_left', '_right'))

,key,value_left,value_right
0,1,A1,B1


<div class="alert alert-info">
<h3> Your turn</h3>
Use the data files located in the folder <TT>../data/FRED</TT> to perform the following tasks:
<ol>
    <li>Load the data in <TT>CPI.csv</TT> and <TT>GDP.csv</TT> into two different DataFrames.
        The files contain monthly data for the Consumer Price Index (CPI) and quarterly data for GDP, respectively.
        <br/>
        <i>Hint:</i> Use <TT>pd.read_csv(..., parse_dates=['DATE'])</TT> to automatically parse strings stored in the <TT>DATE</TT> column as dates.
        </li>
    <li>Merge the CPI with the GDP time series with 
    <a href="https://pandas.pydata.org/docs/reference/api/pandas.DataFrame.merge.html"><TT>merge()</TT></a> 
    using a left join (<TT>how='left'</TT>). How many observations does the resulting DataFrame have?</li>
    <li>Merge the CPI with the GDP time series with <TT>merge()</TT> using an inner join (<TT>how='inner'</TT>). How many observations does the resulting DataFrame have,
        and why is this different from the previous case?</li>
</ol>
</div>

In [23]:
DATA_PATH
cpi = pd.read_csv(f'{DATA_PATH}/CPI.csv', parse_dates=['DATE'])
gdp = pd.read_csv(f'{DATA_PATH}/GDP.csv', parse_dates=['DATE'])


In [24]:
pd.merge(cpi, gdp, on='DATE', how='left')

,DATE,CPI,GDP
0,1947-01-01,21.5,2182.7
1,1947-02-01,21.6,NaN
2,1947-03-01,22.0,NaN
3,1947-04-01,22.0,2176.9
4,1947-05-01,22.0,NaN
...,...,...,...
927,2024-04-01,313.2,23223.9
928,2024-05-01,313.2,NaN
929,2024-06-01,313.0,NaN
930,2024-07-01,313.5,NaN


In [25]:
pd.merge(cpi, gdp, on='DATE', how='inner')

,DATE,CPI,GDP
0,1947-01-01,21.5,2182.7
1,1947-04-01,22.0,2176.9
2,1947-07-01,22.2,2172.4
3,1947-10-01,22.9,2206.5
4,1948-01-01,23.7,2239.7
...,...,...,...
305,2023-04-01,303.0,22539.4
306,2023-07-01,304.6,22780.9
307,2023-10-01,307.5,22960.6
308,2024-01-01,309.7,23053.5


***
### Joining with `join()`

The `DataFrame` method 
[`join()`](https://pandas.pydata.org/docs/reference/api/pandas.DataFrame.join.html) 
is a convenience wrapper around 
[`pd.merge()`](https://pandas.pydata.org/docs/reference/api/pandas.merge.html):

1.  `join()` can be called _only_ directly on the `DataFrame` object
2.  `join()` always operates on the _index_ of the other `DataFrame`
3.  `join()` by default performs a `left` join, whereas `merge()` performs an `inner` join

Use `join()` if you want to join DataFrames which have a similar index.

*Example: joining DataFrames*

- Create two demo DataFrames with keys as index
- `join()` with `how='left'` and `how='inner'`

In [26]:
df_a = df_a.set_index('key')
df_b = df_b.set_index('key')

In [27]:
df_a.join(df_b, on='key', rsuffix='_right', how='outer')

,key,value,value_right
0.0,0,A0,NaN
1.0,1,A1,B1
NaN,2,NaN,B2


<div class="alert alert-info">
<h3> Your turn</h3>
Use the data files located in the folder <TT>../data/FRED</TT> to perform the following tasks:
<ol>
    <li>Load the data in <TT>CPI.csv</TT> and <TT>GDP.csv</TT> into two different DataFrames.
        The files contain monthly data for the Consumer Price Index (CPI) and quarterly data for GDP, respectively.
        <br/>
        <i>Hint:</i> Use <TT>pd.read_csv(..., parse_dates=['DATE'])</TT> to automatically parse strings stored in the <TT>DATE</TT> column as dates.
        </li>
    <li>Set the <TT>DATE</TT> column as the index for each of the two DataFrames.</li>
    <li>Merge the CPI with the GDP time series with 
    <a href="https://pandas.pydata.org/docs/reference/api/pandas.DataFrame.join.html"><TT>join()</TT></a>. 
    Do this with both a left and an inner join.</li>
</ol>
</div>

In [28]:
DATA_PATH
cpi = pd.read_csv(f'{DATA_PATH}/CPI.csv', parse_dates=['DATE'])
cpi = cpi.set_index('DATE')
gdp = pd.read_csv(f'{DATA_PATH}/GDP.csv', parse_dates=['DATE'])
gdp = gdp.set_index('DATE')


In [29]:
cpi.join(gdp, on='DATE', how='left')

,CPI,GDP
DATE,,
1947-01-01,21.5,2182.7
1947-02-01,21.6,NaN
1947-03-01,22.0,NaN
1947-04-01,22.0,2176.9
1947-05-01,22.0,NaN
...,...,...
2024-04-01,313.2,23223.9
2024-05-01,313.2,NaN
2024-06-01,313.0,NaN


In [30]:
cpi.join(gdp, on='DATE', how='inner')

,CPI,GDP
DATE,,
1947-01-01,21.5,2182.7
1947-04-01,22.0,2176.9
1947-07-01,22.2,2172.4
1947-10-01,22.9,2206.5
1948-01-01,23.7,2239.7
...,...,...
2023-04-01,303.0,22539.4
2023-07-01,304.6,22780.9
2023-10-01,307.5,22960.6


***
# Dealing with missing values

- Often arise if concatenated / merged data sets don't contain the same columns or indices

## Dropping missing values

Missing values can be dropped by either

1. Using [`dropna()`](https://pandas.pydata.org/docs/reference/api/pandas.DataFrame.dropna.html)
    or selecting a subset of observations with a boolean operation such as 
    [`notna()`](https://pandas.pydata.org/docs/reference/api/pandas.DataFrame.notna.html).
2. Avoiding the missing values in the first place, e.g., by using `merge(..., how='inner')`.

*Example: Dropping missing values*

- Use _outer/left/right join_ and drop missing data with `dropna()`

*Example: Avoiding missing values in the first place*

- E.g., use _inner join_ when merging data sets

## Filling missing values

Instead of dropping data, we can impute missing values in various ways:

1.  [`fillna()`](https://pandas.pydata.org/docs/reference/api/pandas.DataFrame.fillna.html): replace missing data with user-specified values
2.  [`ffill()`](https://pandas.pydata.org/docs/reference/api/pandas.DataFrame.ffill.html) and 
    [`bfill()`](https://pandas.pydata.org/docs/reference/api/pandas.DataFrame.bfill.html): fill missing values
    forward or backward from adjacent non-missing observations
3.  [`interpolate()`](https://pandas.pydata.org/docs/reference/api/pandas.DataFrame.interpolate.html): interpolate missing values based on non-missing ones

*Example: Replacing missing values with `fillna()`*

- Specify one value for _entire_ `DataFrame`
- Specify different value for each column

In [31]:
df_a = pd.DataFrame({'key': [0, 1], 'value_a' : ['A0', 'A1']})
df_b = pd.DataFrame({'key': [1, 2], 'value_b' : ['B1', 'B2']})

In [32]:
df = pd.merge(df_a, df_b, how='outer', on='key')

In [33]:
df

,key,value_a,value_b
0,0,A0,NaN
1,1,A1,B1
2,2,NaN,B2


In [34]:
df.fillna({'value_a': 'Missing A', 'value_b': 'Missing B'})

,key,value_a,value_b
0,0,A0,Missing B
1,1,A1,B1
2,2,Missing A,B2


*Example: forward- or backward-filling missing values*

- Use [`ffill()`](https://pandas.pydata.org/docs/reference/api/pandas.DataFrame.ffill.html) to forward-fill
- Use [`bfill()`](https://pandas.pydata.org/docs/reference/api/pandas.DataFrame.bfill.html) to backward-fill

In [35]:
df.ffill()

,key,value_a,value_b
0,0,A0,NaN
1,1,A1,B1
2,2,A1,B2


In [36]:
df.bfill()

,key,value_a,value_b
0,0,A0,B1
1,1,A1,B1
2,2,NaN,B2


*Example: linear interpolation*

- Use [`interpolate()`](https://pandas.pydata.org/docs/reference/api/pandas.Series.interpolate.html) to interpolate missing _numerical_ data

In [37]:
s = pd.Series([1.0, 2.0, 3.0, np.nan, 5.0])

In [38]:
s

0    1.0
1    2.0
2    3.0
3    NaN
4    5.0
dtype: float64

In [39]:
s.interpolate()

0    1.0
1    2.0
2    3.0
3    4.0
4    5.0
dtype: float64

<div class="alert alert-info">
<h3> Your turn</h3>
Use the data files located in the folder <TT>../data/FRED</TT> to perform the following tasks:
<ol>
    <li>Load the data in <TT>CPI.csv</TT> and <TT>GDP.csv</TT> into two different DataFrames.
        The files contain monthly data for the Consumer Price Index (CPI) and quarterly data for GDP, respectively.
        <br/>
        <i>Hint:</i> Use <TT>pd.read_csv(..., parse_dates=['DATE'])</TT> to automatically parse strings stored in the <TT>DATE</TT> column as dates.
        </li>
    <li>Merge the CPI with the GDP time series with <TT>merge()</TT> using a left join. This creates missing values in the <TT>GDP</TT>
    column.</li>
    <li>Impute the missing GDP values using <a href="https://pandas.pydata.org/docs/reference/api/pandas.Series.interpolate.html"><TT>interpolate()</TT></a> 
    and replace the missing values in column <TT>GDP</TT>.</li>
</ol>
</div>

In [40]:
df = cpi.join(gdp)

In [41]:
df.interpolate()

,CPI,GDP
DATE,,
1947-01-01,21.5,2182.700000
1947-02-01,21.6,2180.766667
1947-03-01,22.0,2178.833333
1947-04-01,22.0,2176.900000
1947-05-01,22.0,2175.400000
...,...,...
2024-04-01,313.2,23223.900000
2024-05-01,313.2,23223.900000
2024-06-01,313.0,23223.900000
